In [ ]:
import yfinance as yf
import numpy as np
import datetime
import plotly.graph_objs as go
from scipy.optimize import brentq
from scipy.stats import norm
from scipy.interpolate import griddata
import ipywidgets as widgets
from IPython.display import display, clear_output

# Black-Scholes formula to calculate the option price
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

# Function to calculate implied volatility using Black-Scholes formula
def implied_volatility(option_price, S, K, T, r, option_type='call'):
    def objective_function(sigma):
        if option_type == 'call':
            return black_scholes_call(S, K, T, r, sigma) - option_price
    try:
        # Use Brent's method to find the root (implied volatility)
        implied_vol = brentq(objective_function, 0.0001, 5.0)
        return implied_vol
    except ValueError:
        # Return NaN if implied volatility cannot be calculated
        return np.nan

# Fetch and filter options data
def get_filtered_options_data(ticker_symbol, expiration):
    ticker = yf.Ticker(ticker_symbol)
    options_chain = ticker.option_chain(expiration)
    calls = options_chain.calls

    # Filter for reasonable bid-ask spreads and positive prices
    filtered_calls = calls[
        (calls['bid'] > 0) &
        (calls['ask'] > 0) &
        ((calls['ask'] - calls['bid']) < 0.10)
    ]
    return filtered_calls

# Plot the volatility surface
def plot_volatility_surface(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    expiration_dates = ticker.options[:10]  # Use the first 10 available expiration dates

    # Get current stock price and interest rate (set manually here, or fetched from FRED API)
    S = ticker.history(period="1d")['Close'].iloc[-1]
    r = 0.05  # Example: 5% interest rate; could fetch this using FRED API

    # Prepare data for the volatility surface plot
    strike_data = []
    T_data = []
    implied_vol_data = []

    today = datetime.date.today()

    # Iterate over multiple expiration dates
    for expiration in expiration_dates:
        # Fetch filtered options data
        filtered_calls = get_filtered_options_data(ticker_symbol, expiration)

        # Calculate time to expiration (T)
        expiration_date = datetime.datetime.strptime(expiration, "%Y-%m-%d").date()
        T = (expiration_date - today).days / 365  # Convert to fraction of a year

        strikes = filtered_calls['strike'].values
        market_prices = filtered_calls['lastPrice'].values

        # Calculate implied volatilities
        for i in range(len(strikes)):
            iv = implied_volatility(market_prices[i], S, strikes[i], T, r)
            if not np.isnan(iv) and 0.05 < iv < 2.0:  # Filter extreme implied volatilities
                strike_data.append(strikes[i])
                T_data.append(T)
                implied_vol_data.append(iv)

    # Convert lists to numpy arrays for 3D plotting
    strike_data = np.array(strike_data)
    T_data = np.array(T_data)
    implied_vol_data = np.array(implied_vol_data)

    # Restrict grid to actual strike prices from the data
    unique_strikes = np.unique(strike_data)  # Use unique strike prices
    unique_T = np.linspace(min(T_data), max(T_data), 50)  # Keep time to expiration continuous

    # Create grid for strikes and time to expiration
    strike_grid, T_grid = np.meshgrid(unique_strikes, unique_T)

    # Interpolate implied volatility data onto the grid
    implied_vol_grid = griddata(
        (strike_data, T_data),  # Points
        implied_vol_data,       # Values
        (strike_grid, T_grid),  # Grid
        method='linear'         # Change to linear interpolation for better surface stability
    )

    # Create an interactive 3D surface plot using plotly
    fig = go.Figure(data=[go.Surface(
        z=implied_vol_grid, 
        x=strike_grid, 
        y=T_grid,
        colorscale='Viridis'
    )])

    # Customize layout
    fig.update_layout(
        title=f'Volatility Surface for {ticker_symbol.upper()} Options',
        scene=dict(
            xaxis_title='Strike Price',
            yaxis_title='Time to Expiration (Years)',
            zaxis_title='Implied Volatility'
        ),
        autosize=True,
        margin=dict(l=65, r=50, b=65, t=90)
    )

    # Clear the plot area and display the new plot
    with plot_output:
        clear_output(wait=True)
        display(fig)

# Create the input widget (textbox) and button
ticker_input = widgets.Text(
    value='AAPL',  # Default value
    description='Ticker:',
    disabled=False
)

plot_button = widgets.Button(
    description='Plot Volatility Surface',
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to plot the volatility surface'
)

# Create an output widget to display the plot
plot_output = widgets.Output()

# Function to handle button click event
def on_button_clicked(b):
    ticker_symbol = ticker_input.value
    plot_volatility_surface(ticker_symbol)

# Link the button click event to the function
plot_button.on_click(on_button_clicked)

# Display the textbox, button, and output area
display(ticker_input)
display(plot_button)
display(plot_output)
